In [ ]:
# Parameters cell - will be replaced by Papermill
mlflow_tracking_server_arn = None

In [ ]:
import boto3
import mlflow
from mlflow import MlflowClient
import numpy as np
from sagemaker.serve import SchemaBuilder, ModelBuilder
from sagemaker.serve.mode.function_pointers import Mode

print("🚀 Starting Model Deployment and Inference")

In [ ]:
from sagemaker_studio import Project

# Get IAM role from connection
proj = Project()
iam_conn = proj.connection('default.iam')
role = iam_conn.iam_role
print(f"✅ IAM Role: {role}")

# Validate MLflow ARN
if not mlflow_tracking_server_arn:
    raise ValueError("mlflow_tracking_server_arn parameter is required")

print(f"✅ MLflow ARN: {mlflow_tracking_server_arn}")

In [ ]:
# Connect to MLflow and retrieve trained model
print("\n📦 Retrieving model from MLflow...")

mlflow.set_tracking_uri(mlflow_tracking_server_arn)
client = MlflowClient()

model_name = "realistic-classifier-v1"
registered_model = client.get_registered_model(name=model_name)

# Get champion model version
champion_version = client.get_model_version_by_alias(model_name, "champion")
source_path = champion_version.source

print(f"✅ Model: {model_name}")
print(f"✅ Version: {champion_version.version}")
print(f"✅ Source: {source_path}")

In [ ]:
# Define model schema
print("\n🔧 Building model schema...")

sklearn_input = np.random.randn(1, 20)  # 20 features
sklearn_output = 0  # Classification output

schema_builder = SchemaBuilder(
    sample_input=sklearn_input,
    sample_output=sklearn_output,
)

print("✅ Schema built")

In [ ]:
# Build model for deployment
print("\n🏗️  Building model for deployment...")

model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    schema_builder=schema_builder,
    role_arn=role,
    model_metadata={"MLFLOW_MODEL_PATH": source_path},
)

built_model = model_builder.build()
print("✅ Model built")

In [ ]:
# Deploy to endpoint
print("\n🚀 Deploying model to SageMaker endpoint...")

predictor = built_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

endpoint_name = predictor.endpoint_name
print(f"✅ Endpoint deployed: {endpoint_name}")

In [ ]:
# Run inference
print("\n🔮 Running inference...")

test_input = np.random.randn(1, 20)
prediction = predictor.predict(test_input)

print(f"✅ Prediction: {prediction}")
print("✅ Inference successful")

In [ ]:
# Cleanup resources
print("\n🧹 Cleaning up resources...")

try:
    predictor.delete_endpoint(delete_endpoint_config=True)
    print(f"✅ Deleted endpoint: {endpoint_name}")
except Exception as e:
    print(f"⚠️  Cleanup error: {e}")

print("\n✅ Deployment and inference workflow completed successfully!")